In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns # For elbow method visualisation
import geopandas as gpd

import statsmodels.formula.api as smf
import statsmodels
from sklearn.preprocessing import MinMaxScaler

import sklearn.cluster as sklc  # For clustering
import sklearn.metrics as sklm  # For the silhouette score
from sklearn.preprocessing import RobustScaler
import sklearn
from sklearn.preprocessing import MinMaxScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# for plotting
!pip install legendgram
import pysal as p
# import mapclassify as mc
import palettable.matplotlib as palmpl
from legendgram import legendgram

In [ ]:
import re

In [ ]:
ld = pd.read_csv("/kaggle/input/restaurants-and-venues-in-london/London_df.csv")
hyg = pd.read_csv("/kaggle/input/uk-food-hygiene-rating-data-london/food_hygiene_rating_data.csv")
ld_borough = gpd.read_file("/kaggle/input/lodon-boroughs/Boroughs.gpkg")

In [ ]:
ld

In [ ]:
ld["pop_dens"] = ld.Population/ld.Area
ld = ld.drop(['Indian_population','Lat','Lng','Population', 'Dist_from_center'], axis = 1)
ld.rename(columns={"Borough":"LocalAuthorityName"},inplace=True) 
ld.head()

### drop NA (3000+ lines including str in col rating (int type))

In [ ]:
hyg = gpd.GeoDataFrame(hyg, geometry=gpd.points_from_xy(hyg.Longitude, hyg.Latitude, crs='epsg:27700'))
print(hyg.geometry.crs)
print(hyg.total_bounds)
# hyg['RatingValue'] = hyg['RatingValue'].astype('int32')
hyg_with_rating = hyg.copy()
hyg_with_rating['RatingValue'] = hyg_with_rating['RatingValue'].apply(lambda x: x if re.search("^\d+$", str(x)) else np.nan)
hyg_with_rating = hyg_with_rating.dropna()
hyg_with_rating.RatingValue.astype({'RatingValue': 'int32'}).dtypes # convert rating type to int 

Trying to plot London boroughs with points from hyg

In [ ]:
# q = mc.Quantiles(hyg_with_rating.RatingValue, k=5) # For Fisher Jenks you may want 'Sampled' to speed things up

f,ax = plt.subplots(figsize=(15,9))
hyg_with_rating.plot(column='RatingValue',   marker='*', markersize=0.5, ax=ax)
ld_borough.plot(edgecolor=(0.8, 0, 0, 0.5), facecolor='none', linewidth=2.5, ax=ax)

# ax.set_xlim([525000,540000])
# ax.set_ylim([176000,186000])
ax.set_xlim([500000,570000])
ax.set_ylim([150000,220000])
# ax.axis('off') #don't plot the axes (bounding box)
# ax.set_title('London Shop Ratings', fontdict={'fontsize':'20', 'fontweight':'3'})  #provide a title
# a = ax.annotate('Source: /kaggle/input/uk-food-hygiene-rating-data-london',
#              xy=(0.905, 0.055), xycoords='figure fraction', 
#              horizontalalignment='right', verticalalignment='bottom', 
#              fontsize=12, fontweight=4, color='#555555')  #add source info on the image itself
# a.set_bbox(dict(facecolor='white', alpha=0.95, edgecolor='white'))
# ax.get_legend().set_title("RatingValue")  #set the legend title

# legendgram(f, ax, 
#             hyg_with_rating.RatingValue, q.bins, bins=round(hyg_with_rating.RatingValue.max()/25),
#             pal=palmpl.Viridis_5,
#             legend_size=(0.3, 0.1), 
#             loc='lower left',
#             clip=(0,750),
#             frameon=True
#     )

# f.savefig('ppn.png', dpi=150)

In [ ]:
hyg_with_rating.plot(column='RatingValue',   marker='*', markersize=0.5)

### Select coords of City of London as centre

In [ ]:
from shapely import geometry
# to_select = ld_borough["NAME"].str.contains("City of London")
# centre = ld_borough[to_select].centroid
centre = gpd.GeoSeries([geometry.Point(-0.09742999999997437, 51.5205000000000)]*len(hyg.geometry),              
                         crs='EPSG:27700'  
                         )

In [ ]:
centre

In [ ]:
hyg['dist_from_centre'] = hyg.geometry.distance(centre)

In [ ]:
hyg.info()

In [ ]:
ld.info()

In [ ]:
# hyg = hyg.join(ld, on = "LocalAuthorityName", how = "left")
# hyg = pd.concat(hyg.drop('geometry',axis = 1), ld)

### Problems occurred when joining. Trying another method

In [ ]:
dic = {}
for i in range(ld.shape[0]):
    dic[ld.values[i][0]] = ld.values[i][1]
# dic[ld[0][0]] = ld[0][1]

In [ ]:
dic['Barking and Dagenham']=dic['Barking and Dagenham ']
del dic['Barking and Dagenham ']
dic['Greenwich']=dic['Greenwich ']
del dic['Greenwich ']
dic['Kingston-Upon-Thames']=dic['Kingston upon Thames']
del dic['Kingston upon Thames']
dic['Hammersmith and Fulham']=dic['Hammersmith and Fulham ']
del dic['Hammersmith and Fulham ']
dic['City of London Corporation']=dic['City of London']
dic

In [ ]:
hyg.LocalAuthorityName.values

In [ ]:
hyg['PopDens'] = hyg['LocalAuthorityName'].apply(lambda x: dic[x])

### Down-sampling

In [ ]:
sliced = hyg.sample(n = 1000, random_state=42).dropna(axis = 'index').reset_index(drop=True) # don't forget to dropna before re-indexing


In [ ]:
sliced

In [ ]:
%%time
'''
Gets count of businesses of the same category in a proximity of 'thres'

'''
def get_rivals(df, col, thres = 0.03): 
    df['buffer'] = df[col].buffer(thres)
    rivals = [None]*df.shape[0]
    for i in range(0, df.shape[0]):
        cnt = 0
        for j in range(0, df.shape[0]):
            if df['buffer'][i].covers(df.geometry[j]) and df.BusinessType[i] == df.BusinessType[j]:
                cnt += 1
            rivals[i] = cnt - 1 # subtracts itself
    df["rivals"] = rivals
    return df
# sliced = hyg[0:500]
sliced = get_rivals(sliced, "geometry", thres = 0.025)
sliced.describe()
# hyg = get_rivals(hyg, "geometry", thres = 0.0)
# hyg.describe()

In [ ]:
hyg

In [ ]:
sliced.drop("Unnamed: 0", axis = 1, inplace = True)
sliced.to_csv("hyg_dist_PopDens_rivals_added.csv")

In [ ]:
sliced

# Regression

In [ ]:
sliced.corr() 

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(sliced.corr(), fignum=f.number)
plt.xticks(range(sliced.select_dtypes(['number']).shape[1]), sliced.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(sliced.select_dtypes(['number']).shape[1]), sliced.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In [ ]:
reg_rivals_dist_popDens = smf.ols(formula='rivals ~ dist_from_centre + PopDens',data=sliced).fit()

# And this gives a big summary of the results:
print(reg_rivals_dist_popDens.__class__.__name__)
print(reg_rivals_dist_popDens.summary())

reg_rivals_dist_popDens.params

In [ ]:
plt.scatter(reg_rivals_dist_popDens.fittedvalues, reg_rivals_dist_popDens.resid)
# adding title and labels
plt.xlabel('Fitted SQ')
plt.ylabel('Residual Profit')
plt.title('Residual vs. Fitted Plot (of retail profit)')
plt.show()

# Clustering  
K-means is susceptible to outliers. So we exclude them first.

In [ ]:
hyg = hyg[hyg.dist_from_centre <= hyg.dist_from_centre.quantile(q=0.99)]

In [ ]:
hyg.shape

In [ ]:
# def filter_ab(df):
#     r = df.rolling(window=20, center = True)
#     print(r.std())
#     mp1 = r.mean() + 1. * r.std()
#     mp2 = r.mean() - 1. * r.std()
#     return df.where( (df < mp1) & (df > mp2) ) 
# to_exclude_ab = hyg.loc[:,['dist_from_centre','PopDens']]
# to_exclude_ab = to_exclude_ab.apply(filter_ab)
# to_exclude_ab

## Deciding on the number of clusters using elbow method

In [ ]:
def elbow_vis(df, x, y):
#     from sklearn.preprocessing import MinMaxScaler
    df = df.loc[:,[x, y]]
    mms = MinMaxScaler()
    data_standardised = df.copy()
    for c in df.columns.values:
        data_standardised[c] = mms.fit_transform(df[c].values.reshape(-1,1))
        
    wcss = []
    for i in range(1, 11):
        clustering = sklc.KMeans(n_clusters=i, init='k-means++', random_state=42)
        clustering.fit(data_standardised)
        wcss.append(clustering.inertia_)
    
    ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    sns.lineplot(x = ks, y = wcss)

In [ ]:
elbow_vis(hyg, 'PopDens', 'dist_from_centre')

In [ ]:
def Kmeans_2dim(df, x, y, num_clusters = 3, random_state_seed = 100):
#     from sklearn.preprocessing import MinMaxScaler
    df = df.loc[:,[x, y]]
    # to_stdise

    # This line sets the number of clusters you want to find:
    num_clusters = num_clusters

    #Data standardisation
    
    mms = MinMaxScaler()

    data_standardised = df.copy()
    for c in df.columns.values:
        data_standardised[c] = mms.fit_transform(df[c].values.reshape(-1,1)) #
#     data_standardised.head()

    # we fix the random_state so that the kmeans result is reproducible
    kmeans_output = sklc.KMeans(n_clusters=num_clusters, random_state=random_state_seed).fit(data_standardised)
    clustering_ids_kmeans = kmeans_output.labels_
    # Sanity check
#     print(kmeans_output)

    # we will combine the clustering IDs to the dataframe
    data = data_standardised.assign(cluster_id = clustering_ids_kmeans)
    
    # plot
    fig, ax = plt.subplots(figsize=(12,8))

    # number of clusters
    n_clusters = data.cluster_id.nunique()

    # get discrete colormap
    cmap = plt.get_cmap('Spectral', n_clusters)

    # scatter points
    scatter = ax.scatter(data[x], data[y], c=data.cluster_id, cmap=cmap)
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_title('Clustering result x - y')

    # add colorbar
    cbar = plt.colorbar(scatter)

    # set ticks locations (not very elegant, but it works):
    # - shift by 0.5
    # - scale so that the last value is at the center of the last color
    tick_locs = (np.arange(n_clusters) + 0.5)*(n_clusters-1)/n_clusters
    cbar.set_ticks(tick_locs)

    # set tick labels (as before)
    cbar.set_ticklabels(np.arange(n_clusters))
    silhouette_kmeans = sklm.silhouette_score(data, clustering_ids_kmeans)
    # Silouette score is a measure of how well the data is clustered, from -1 (extremely poor) to 1 (extremely strong).
    print("Silhouette Score:", silhouette_kmeans)
    
    
        
    return data

In [ ]:
hyg_clu = Kmeans_2dim(hyg, 'PopDens', 'dist_from_centre', num_clusters = 3)

hyg_clu = Kmeans_2dim(hyg, 'PopDens', 'dist_from_centre', num_clusters = 4)

In [ ]:
hyg_clu = Kmeans_2dim(hyg, 'PopDens', 'dist_from_centre', num_clusters = 5)

In [ ]:
clustered = pd.concat([hyg_clu, hyg.RatingValue], axis=1)

In [ ]:
clustered

In [ ]:
a = hyg_clu.groupby(['cluster_id', 'SubCategory']).count().sort_values(by = ['cluster_id', 'Profit'],ascending = False)
a.rename(columns={"PQ":"count"}, inplace=True) 
a.drop('Profit', axis = 1, inplace=True)
a